In [1]:
# -*- coding: utf8 -*-


from sirsaia import brasilio
from sirsaia import epiestim
from sirsaia import plotutils


import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

/home/flaviovdf/miniconda3/envs/corona/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
plt.ion()
plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

In [3]:
config = epiestim.make_config(mean_si=5, std_si=2, si_parametric_distr='weibull')

In [4]:
df_orig = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
brazil = df_orig[df_orig['Country/Region'] == 'Brazil'].T
brazil = brazil[4:].copy()
brazil.index = pd.to_datetime(brazil.index)
brazil = brazil.sort_index().astype('i').diff().fillna(0)
first_day = np.nonzero(brazil.values)[0][0]
brazil = brazil[first_day:]
brazil.columns = ['local']
epiestim_result = epiestim.estimate_r(brazil, config)
plt.title('Brazil (last mean = {})'.format(epiestim_result['Mean(R)'].values[-1]))
plotutils.plot_result(epiestim_result, brazil)
plt.tight_layout()
plt.savefig('plots/Brasil.png')
plt.close()

/home/flaviovdf/miniconda3/envs/corona/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Default config will estimate R on weekly sliding windows.
    To change this change the t_start and t_end arguments. 

  warnings.warn(x, RRuntimeWarning)


In [5]:
datasets = {}
sheets = glob.glob('data/covid19/[A-Z]*/*.xlsx')
for sheet in sorted(sheets):
    state = os.path.basename(sheet)[:2]
    region = os.path.basename(os.path.dirname(sheet))
    if region not in datasets:
        datasets[region] = {}
    datasets[region][state] = sheet

In [6]:
pop = pd.read_csv('data/populacao-estimada-2019.csv')

In [7]:
for region in datasets:
    for state, fpath in datasets[region].items():
        df = brasilio.load_data(fpath)
        model_df = df[['TOTAL NO ESTADO', 'Importados/Indefinidos']].copy()
        model_df.columns = ['local', 'imported']
        epiestim_result = epiestim.estimate_r(model_df, config)
        plotutils.plot_result(epiestim_result, model_df)
        plt.title(state + ' last mean = {}'.format(epiestim_result['Mean(R)'].values[-1]))
        try:
            os.makedirs('plots/{}/'.format(region))
        except:
            pass
        plt.tight_layout()
        plt.savefig('plots/{}/{}.png'.format(region, state))
        plt.close()
        
        top_5 = pop[pop['uf'] == state].sort_values(by='populacao_estimada')[-5:]['municipio']
        for city in top_5.values:
            model_df = df[['TOTAL NO ESTADO']].copy()
            model_df.columns = ['local']
            epiestim_result = epiestim.estimate_r(model_df, config)
            plotutils.plot_result(epiestim_result, model_df)
            plt.title(state + '-' + city + ' last mean = {}'.format(epiestim_result['Mean(R)'].values[-1]))
            plt.tight_layout()
            plt.savefig('plots/{}/{}-{}.png'.format(region, state, city))
            plt.close()municipio

SyntaxError: invalid syntax (<ipython-input-7-9a1fcedea09e>, line 26)

In [ ]:
df = brasilio.load_data('data/covid19/Sudeste/MG.xlsx')
model_df = df[['Belo Horizonte']].copy()
model_df.columns = ['local']
epiestim_result = epiestim.estimate_r(model_df)
plt.title('Belo Horizonte (R(t) 31/03 = {})'.format(epiestim_result['Mean(R)'].values[-1]))
plotutils.plot_result(epiestim_result, model_df)

In [ ]:
epiestim_result